# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [25]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
import numpy as np
from scipy.sparse import lil_matrix
from collections import Counter

In [26]:
news = open(r"C:\Users\marus\Documents\Вышка\compling_nlp_hse_course\notebooks\lm_intro\lenta.txt", encoding = 'utf-8').read()

### Генератор 
NOTE: По какой-то причине в гитхабе не отображаются теги start и end - ниже везде, где кавычки с пустотой, стоит один из этих тегов

In [27]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20]
    
    return normalized_text

In [28]:
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [29]:
# Функция для подсчета n-грамм

def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [30]:
# Униграммы, биграммы и триграммы для новостей

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, 2))
    trigrams_news.update(ngrammer(sentence, 3))

In [31]:
# Заполняем матрицу

matrix_news = lil_matrix((len(bigrams_news), 
                         len(unigrams_news)))

id2word_news_1 = list(unigrams_news)
word2id_news_1 = {word:i for i, word in enumerate(id2word_news_1)}
id2word_news_2 = list(bigrams_news)
word2id_news_2 = {word:i for i, word in enumerate(id2word_news_2)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    unigram = word3
    matrix_news[word2id_news_2[bigram], word2id_news_1[unigram]] =  (trigrams_news[ngram]/bigrams_news[bigram])

In [32]:
# Функция для генерации текстов

def generate(matrix, id2word_1, word2id_2, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id_2[start]
    current_bigram = start

    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word_1[chosen])
        current_bigram = f'{current_bigram.split()[1]} {id2word_1[chosen]}'
        
        if id2word_1[chosen] == '<end>':
            chosen = word2id_2[start]
            current_bigram = start
            
        current_idx = word2id_2[current_bigram]

    return ' '.join(text)

In [39]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

в то же время такие темпы роста населения планеты продолжают увеличиваться 
 мы не будем немедленно делать вывод о целесообразности размещения подобных табло на других вокзалах ирландии глава всероссийского центра изучения общественного мнения на различные актуальные темы 
 он признал что его фирма mabetex действительно помогала ельцину и его структурные службы увд центрального северо-восточного южного административных округов москвы города зеленограда а также начало конца старых масс-медиа а также населенных пунктов либо разрушены либо отрезаны стихией от внешнего мира 
 в этой связи нил аберкромби отметил построение в россии связана с тем как сообщает риа новости 
 в то время как


### Перплексия

In [59]:
# Отложенные предложения из корпуса

probas_news = [normalize(text) for text in sent_tokenize(news[-3000:])][0:39]

In [60]:
# Сгенерированные тексты
text_generated = generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n').split('\n')[0:3]

In [61]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

# Перплексия на корпусе
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

# перплексия на биграмной модели

def compute_join_proba_markov_assumption_bigrams(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 2):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

# перплексия на триграмной модели

def compute_join_proba_markov_assumption_trigrams(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 3):
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        unigram = word3
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [62]:
perplx1 = [perplexity(*compute_joint_proba(i, probas_news)) for i in text_generated]
perplx2 = [perplexity(*compute_join_proba_markov_assumption_bigrams(i, unigrams_news, bigrams_news)) for i in text_generated]
perplx3 = [perplexity(*compute_join_proba_markov_assumption_trigrams(i, bigrams_news, trigrams_news)) for i in text_generated]

In [63]:
print(f'Средняя перплексия униграмной модели - {np.mean(perplx1)}')
print(f'Средняя перплексия биграмной модели - {np.mean(perplx2)}')
print(f'Средняя перплексия триграмной модели - {np.mean(perplx3)}')

Средняя перплексия униграмной модели - 5000.000000000006
Средняя перплексия биграмной модели - 51.27200325900432
Средняя перплексия триграмной модели - 2.9045891985380035


Ожидаемо, модель на биграммах более точна, чем модель на униграммах, а модель на триграммах - точнее и униграмной, и биграмной.

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Незнакомые слова могут встретиться в открытой лексической системе. Чтобы модель работала с потенциально незнакомыми для себя словами, эти слова моделируются с помощью псевдослова <UNK>, вероятность которого можно посчитать двумя способами:
1. Выбрать заранее фиксированный список слов. Все слова, которые в него не входят, заменить на <UNK>. Считать <UNK> просто одним из слов и, соответственно, рассчитывать ему вероятность так же, как и другим обычным словам.
    
2. Если нет возможности составить фиксированный список слов, можно заменить на <UNK> слова, которые встречаются реже, чем n раз (n – некое маленькое число, зависит от общего размера словаря) - <UNK> опять же будет рассматриваться как обычное слово.

2. Что такое сглаживание (smoothing)?

Сглаживание - это метод, который используется для того, чтобы избежать нулевой вероятности для тех случаев, когда слово встречается в незнакомом для модели контексте. Для этого часть вероятностной массы забирается от самых частотных событий и резервируется для событий, которые еще не встречались. Благодаря этому такие события получают не нулевую вероятность, а часть от "зарезервированной" вероятности.
